In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import joblib
import optuna
from sklearn.model_selection import cross_val_score

In [ ]:

train = np.load("nạp dữ liệu")
X_train, y_train = train["X"], train["y"]

test = np.load("nạp dữ liệu")
X_test, y_test = test["X"], test["y"]

In [ ]:
def objective(trial):
    # Định nghĩa hàm mục tiêu, nhận một 'trial' đại diện cho lần thử tham số
    n_estimators = trial.suggest_int('n_estimators', 50, 300)  
    # Tối ưu số lượng cây từ 50 đến 300
    max_depth = trial.suggest_int('max_depth', 5, 30)  
    # Tối ưu độ sâu tối đa của cây từ 5 đến 30
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])  
    # Tối ưu số đặc trưng xem xét khi phân chia nút, chọn trong 3 kiểu phổ biến
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)  
    # Tối ưu số mẫu tối thiểu để phân chia một nút
    class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])  
    # Tối ưu class_weight: không dùng hoặc cân bằng trọng số lớp
    
    # Random_state được tối ưu là 42
    random_state = 42
    
    # Tối ưu n_jobs
    n_jobs = -1
    
    # Khởi tạo mô hình RandomForest với các tham số được thử nghiệm
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        min_samples_split=min_samples_split,
        class_weight=class_weight,
        random_state=random_state,
        n_jobs=n_jobs
    )
    
    # Huấn luyện mô hình với tập train
    clf.fit(X_train, y_train)
    # Dự đoán trên tập test
    y_pred = clf.predict(X_test)
    # Trả về accuracy làm metric để tối ưu
    return accuracy_score(y_test, y_pred)

In [ ]:
# Tạo study Optuna, đặt hướng tối ưu là 'maximize' (tăng accuracy)
study = optuna.create_study(direction='maximize')
# Chạy tối ưu hàm objective trong 30 lần thử
study.optimize(objective, n_trials=30)

In [ ]:
# In ra kết quả tốt nhất tìm được sau 30 lần thử
print("Best trial:")
print(f"Accuracy: {study.best_trial.value}")  # Giá trị accuracy cao nhất
print("Params:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")  # In tham số tốt nhất của mô hình

In [ ]:
# Train lại với tham số tốt nhất
best_params = study.best_trial.params
best_rf = RandomForestClassifier(
    **best_params,
    random_state=42,
    n_jobs= -1
)
best_rf.fit(X_train, y_train)

['model_randomforest.joblib']

In [ ]:
# Lưu model
joblib.dump(best_rf, "model_randomforest_optuna_face.joblib")


In [ ]:
# Đánh giá chi tiết mô hình trên tập test
y_pred = best_rf.predict(X_test)
print("\nKết quả mô hình Random Forest tối ưu:")
print("Accuracy:", accuracy_score(y_test, y_pred))  # Độ chính xác
print(classification_report(y_test, y_pred, zero_division=0))  # Báo cáo chi tiết precision, recall,...


# Tạo danh sách các mẫu dự đoán sai
wrong = [(true, pred) for true, pred in zip(y_test, y_pred) if true != pred]
print(f"[!] Số mẫu sai: {len(wrong)}")  # Số mẫu bị phân loại sai
print("Một số lỗi phổ biến:", Counter(wrong).most_common(5))  # 5 lỗi sai phổ biến nhất


 Kết quả mô hình Random Forest:
Accuracy: 0.9951690821256038
               precision    recall  f1-score   support

     Duy Khôi       1.00      1.00      1.00         2
        Dũng       1.00      1.00      1.00         2
      Dương       1.00      1.00      1.00        16
          Hà       1.00      1.00      1.00         2
       Hiếu       0.96      1.00      0.98        24
        Hưng       1.00      1.00      1.00         2
         Khôi       1.00      0.50      0.67         2
        Lành       1.00      1.00      1.00         6
         Linh       1.00      1.00      1.00        36
        Luân       1.00      1.00      1.00         3
       Nghĩa       1.00      1.00      1.00         3
       Nguyên       1.00      1.00      1.00         9
Nhật Tiến       1.00      1.00      1.00         2
         Phú       1.00      1.00      1.00         5
     Phương       1.00      1.00      1.00        18
        Quang       1.00      1.00      1.00        21
  